# Introduction to Automated Machine Learning (AutoML)

This sample shows how you can use AutoML to automate the process of training custom ML models.

In this case, we want to train an ML model that automatically applies a label to GitHub issues.

## Install NuGet packages

In [1]:
#r "nuget: Microsoft.ML.AutoML, 0.22.2"

Installed Packages Microsoft.ML.AutoML, 0.22.2

Loading extensions from `C:\Users\davidm\.nuget\packages\microsoft.ml.automl\0.22.2\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `C:\Users\davidm\.nuget\packages\skiasharp\2.88.8\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

## Add using statements

In [2]:
using System.Threading;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

## Initialize MLContext

In [3]:
// Initialize MLContext
MLContext ctx = new MLContext();

## Use AutoML to infer column information

In [4]:
// Define data path
var dataPath = Path.GetFullPath(@"Data/issues_train.tsv");

// Infer column information
ColumnInferenceResults columnInference =
    ctx.Auto().InferColumns(dataPath, separatorChar: '\t', labelColumnName: "Area", groupColumns: false);

In [5]:
columnInference

Microsoft.ML.AutoML.ColumnInferenceResults TextLoaderOptions Microsoft.ML.Data.TextLoader+Options AllowQuoting True AllowSparse False InputSize <null> Separators [ ] DecimalMarker . Columns index value 0 Microsoft.ML.Data.TextLoader+Column DataKind Single Name ID Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 0 Max 0 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 1 Microsoft.ML.Data.TextLoader+Column DataKind String Name Area Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 1 Max 1 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 2 Microsoft.ML.Data.TextLoader+Column DataKind String Name Title Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 2 Max 2 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 3 Microsoft.ML.Data.TextLoader+Column DataKind String Name Description Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 3 Max 3 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> TrimWhitespace False HasHeader True UseThreads True ReadMultilines False HeaderFile <null> MaxRows <null> EscapeChar " MissingRealsAsNaNs False ColumnInformation Microsoft.ML.AutoML.ColumnInformation LabelColumnName Area UserIdColumnName <null> GroupIdColumnName <null> ItemIdColumnName <null> ExampleWeightColumnName <null> SamplingKeyColumnName <null> CategoricalColumnNames [ ] NumericColumnNames [ ID ] TextColumnNames [ Title, Description ] IgnoredColumnNames [ ] ImagePathColumnNames [ ]

## Load data into IDataView

In [6]:
// Create text loader
TextLoader loader = ctx.Data.CreateTextLoader(columnInference.TextLoaderOptions);

// Load data into IDataView
IDataView data = loader.Load(dataPath);

## Remove columns

In [7]:
var columnsToExclude = new[]{"ID", "Description"};

data = ctx.Transforms.DropColumns(columnsToExclude)
    .Fit(data)
    .Transform(data)

Microsoft.ML.Transforms.ColumnSelectingTransformer+SelectColumnsDataTransform CanShuffle False Source Microsoft.ML.Data.TextLoader+BoundLoader CanShuffle False Schema [ ID: Single, Area: String, Title: String, Description: String ] Count 4 (values) index value 0 ID: Single Name ID Index 0 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 1 Area: String Name Area Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 2 Title: String Name Title Index 2 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 3 Description: String Name Description Index 3 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) InputSchema [ ID: Single, Area: String, Title: String, Description: String ] Count 4 (values) index value 0 ID: Single Name ID Index 0 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 1 Area: String Name Area Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 2 Title: String Name Title Index 2 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 3 Description: String Name Description Index 3 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) OutputSchema [ Area: String, Title: String ] Count 2 (values) index value 0 Area: String Name Area Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 1 Title: String Name Title Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty)

## Split data into train / validation

80% of the dataset is used for training and 20% is used for validation (tuning)

In [8]:
var trainValidationData = ctx.Data.TrainTestSplit(data, testFraction: 0.2);

In [9]:
SweepablePipeline pipeline =
    ctx.Auto().Featurizer(data, columnInformation: columnInference.ColumnInformation)
        .Append(ctx.Transforms.Conversion.MapValueToKey(columnInference.ColumnInformation.LabelColumnName))
        .Append(ctx.Auto().MultiClassification(labelColumnName: columnInference.ColumnInformation.LabelColumnName))
        .Append(ctx.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

## Create AutoML experiment

In [10]:
AutoMLExperiment experiment = ctx.Auto().CreateExperiment();

## Configure experiment settings

In [11]:
experiment
.SetPipeline(pipeline)
.SetMulticlassClassificationMetric(MulticlassClassificationMetric.MacroAccuracy, labelColumn: columnInference.ColumnInformation.LabelColumnName)
.SetTrainingTimeInSeconds(120)
.SetDataset(trainValidationData);

## Configure experiment monitor

In [12]:
public class AutoMLMonitor : IMonitor
{
    private readonly List<TrialResult> _completedTrials;
    private readonly SweepablePipeline _pipeline;

    public AutoMLMonitor(SweepablePipeline pipeline)
    {
        _completedTrials = new List<TrialResult>();
        _pipeline = pipeline;
    }

    public IEnumerable<TrialResult> GetCompletedTrials() => _completedTrials;

    public void ReportBestTrial(TrialResult result)
    {
        return;
    }

    public void ReportCompletedTrial(TrialResult result)
    {
        var trialId = result.TrialSettings.TrialId;
        var timeToTrain = result.DurationInMilliseconds;
        var pipeline = _pipeline.ToString(result.TrialSettings.Parameter);
        Console.WriteLine($"Trial {trialId} finished training in {timeToTrain}ms with pipeline {pipeline}");
        _completedTrials.Add(result);
    }

    public void ReportFailTrial(TrialSettings settings, Exception exception = null)
    {
        if (exception.Message.Contains("Operation was canceled."))
        {
            Console.WriteLine($"{settings.TrialId} cancelled. Time budget exceeded.");
        }
        Console.WriteLine($"{settings.TrialId} failed with exception {exception.Message}");
    }

    public void ReportRunningTrial(TrialSettings setting)
    {
        return;
    }
}

In [13]:
var monitor = new AutoMLMonitor(pipeline);
experiment.SetMonitor(monitor);

## Train the model

In [14]:
var cts = new CancellationTokenSource();
TrialResult experimentResults = await experiment.RunAsync(cts.Token);

Trial 0 finished training in 1479ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 1 finished training in 1066ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 2 finished training in 1376ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown
Trial 3 finished training in 69ms with pipeline FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown
Trial 4 finished training in 1226ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 5 finished training in 1354ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 6 finished training in 1251ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown
Trial 7 finished training in 1038ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 8 finished training in 1122ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown
Trial 9 finished train

## Get the best model

In [15]:
var bestModel = experimentResults.Model;
bestModel

[ Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer, Microsoft.ML.Data.ColumnConcatenatingTransformer, Microsoft.ML.Transforms.ValueToKeyMappingTransformer, Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters], Microsoft.ML.Transforms.KeyToValueMappingTransformer ] LastTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer (values) index type value 0 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 1 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 2 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 3 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName Features FeatureColumnType Vector<Single, 2202> Dimensions [ 2202 ] IsKnownSize True ItemType Single RawType System.Single Size 2202 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.OneVersusAllModelParameters 4 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

## Display the metric for the best model

In [16]:
experimentResults.Metric

0.16666666666666666

## Try out the model

### Make predictions

In [17]:
var predictions = bestModel.Transform(trainValidationData.TestSet);

### Display prediction results

In [18]:
predictions.Preview().RowView

index value 0 8 columns Values index value 0 [Area, Unlabeled] Key Area Value Unlabeled 1 [Area, 3] Key Area Value 3 2 [Title, osx-x64 Release AllSubsets_Mono_Minijit_RuntimeTests minijit build leg fails to build with an error referencing a macosx12.0] Key Title Value osx-x64 Release AllSubsets_Mono_Minijit_RuntimeTests minijit build leg fails to build with an error referencing a macosx12.0 3 [Title, Sparse vector of size 2202, 75 explicit values] Key Title Value [ 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926 ... (more) ] IsDense False Length 2202 (values) [ 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926 ... (more) ] 4 [Features, Sparse vector of size 2202, 75 explicit values] Key Features Value [ 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926 ... (more) ] IsDense False Length 2202 (values) [ 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926, 0.10540926 ... (more) ] 5 [PredictedLabel, 1] Key PredictedLabel Value 1 6 [PredictedLabel, area-System.Threading] Key PredictedLabel Value area-System.Threading 7 [Score, Dense vector of size 30] Key Score Value [ 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335 ... (more) ] IsDense True Length 30 (values) [ 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335, 0.033333335 ... (more) ] 1 8 columns Values index value 0 [Area, area-Build-mono] Key Area Value area-Build-mono 1 [Area, 11] Key Area Value 11 2 [Title, The 'wasi-experimental' workload is not supported in .NET 9] Key Title Value The 'wasi-experimental' workload is not supported in .NET 9 3 [Title, Sparse vector of size 2202, 46 explicit values] Key Title Value [ 0.1490712, 0.1490712, 0.1490712, 0.2981424, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712 ... (more) ] IsDense False Length 2202 (values) [ 0.1490712, 0.1490712, 0.1490712, 0.2981424, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712 ... (more) ] 4 [Features, Sparse vector of size 2202, 46 explicit values] Key Features Value [ 0.1490712, 0.1490712, 0.1490712, 0.2981424, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712 ... (more) ] IsDense False Length 2202 (values) [ 0.1490712, 0.1490712, 0.1490712, 0.2981424, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712, 0.1490712 ... (more) ] 5 [PredictedLabel, 1] Key PredictedLabel Value 1 6 [PredictedLabel, area-System.Threading] Key PredictedLabel Value area-System.Threading 7 [Score, Dense vector of size 30

## Save the best model

In [19]:
ctx.Model.Save(bestModel, data.Schema, "model.mlnet");